In [6]:
! pip install lxml
! pip install html5lib
! pip install BeautifulSoup4
! pip install tabulate

     |████████████████████████████████| 5.8MB 29.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 6.0MB/s ta 0:00:011
     |████████████████████████████████| 51kB 12.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/e1/41/5e/e201f95d90fc84f93aa629b6638adacda680fe63aac47174ab
Successfully built tabulate


In [7]:
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
import requests

#### Read data

In [8]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove rows with Borough = "Not assigned"

In [9]:
df = df[df['Borough'] != 'Not assigned' ]
df.head()
df.shape

(211, 3)

#### Aggregate the data by Postcode & Borough

In [11]:
g_df = df.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda col: ', '.join(col))
g_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Assign Borough to Neighbourhood if not assigned

In [12]:
df_final = g_df
num = df_final.shape[0]
Nghb = [None] * num

for i in range(0, num):
    if df_final.loc[i, 'Neighbourhood']== 'Not assigned':
        Nghb[i]=df_final.loc[i, 'Borough']
    else:
        Nghb[i]=df_final.loc[i, 'Neighbourhood']
df_final['Neighbourhood'] = Nghb


#### Check the final table

In [13]:
df_final.shape

(103, 3)

In [14]:
print(df_final.to_string())

    Postcode           Borough                                      Neighbourhood
0        M1B       Scarborough                                     Rouge, Malvern
1        M1C       Scarborough             Highland Creek, Rouge Hill, Port Union
2        M1E       Scarborough                  Guildwood, Morningside, West Hill
3        M1G       Scarborough                                             Woburn
4        M1H       Scarborough                                          Cedarbrae
5        M1J       Scarborough                                Scarborough Village
6        M1K       Scarborough        East Birchmount Park, Ionview, Kennedy Park
7        M1L       Scarborough                    Clairlea, Golden Mile, Oakridge
8        M1M       Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9        M1N       Scarborough                        Birch Cliff, Cliffside West
10       M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11       M1R    

#### Read the Geospatial data

In [15]:
GeoData = pd.read_csv('http://cocl.us/Geospatial_data')
GeoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge data

In [16]:
df_merge = pd.merge(df_final, GeoData, left_on = 'Postcode', right_on = 'Postal Code')
df_merge = df_merge.drop(['Postal Code'], axis=1)
df_merge.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Explore Neighbourhood

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [3]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinates of Toronto are 43.653963, -79.387207.


### Create Toronto Map

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Postcode'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

#### Look at Toronto only

In [36]:
toronto_only = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_df = df_merge[df_merge['Borough'].isin(toronto_only)].reset_index(drop=True)

#### Using FourSqure API to explore

In [23]:
CLIENT_ID = 'HMM04OVGKUVJBTDUYYOSAEDYSF3TMVQQPOYT5WFAN3QA0UW4' # your Foursquare ID
CLIENT_SECRET = 'BJY5C01WWSJP2DI15TMADEB5YLRSMGF55RG3M4YFWYU1IOAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [37]:

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Postcode'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [38]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1722, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
